# 📂 Imports 📂

In [ ]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import optuna

from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, QuantileTransformer, LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, VotingRegressor, StackingRegressor, BaggingRegressor
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.decomposition import PCA
from sklearn.pipeline import FeatureUnion
from sklearn.inspection import PartialDependenceDisplay
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.base import BaseEstimator, TransformerMixin

from xgboost import XGBRegressor, plot_importance
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

from imblearn.pipeline import Pipeline

from itertools import combinations

# 📈 Exploratory Data Analysis 📊

### Importing Data + a High-Level Look at the Data

In [ ]:
# import the data
all_data = pd.read_csv('data.csv')

# separate data into train and submission sets based on blank target values
train = all_data[all_data['x_e_out [-]'].isna() == False]
submission = all_data[all_data['x_e_out [-]'].isna() == True]

# get length of train and test datasets
print(f'\nTrain dataset length: {train.shape[0]}')
print(f'Submission dataset length: {submission.shape[0]}\n')

# check for missing values
print(f'There are {int(train.isna().sum().sum())} missing feature values in the train set.')
print(f'There are {int(submission.isna().sum().sum())} missing feature values in the submission set.\n')

# check for duplicate rows
n_duplicate_rows = len(train) - len(train.drop_duplicates())
print(f'There are {int(n_duplicate_rows)} duplicate rows in the train dataset.\n')

# quick high-level overview of dataset
pd.set_option('display.expand_frame_repr', False) # need this because there are so many features
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
display(train.head())
print('\n\n')
display(train.describe().round(decimals=2))

#### 💡Insights: First Glance
- Train dataset is approximately twice as large as the submission dataset
- The train dataset is missing a significant amount of data, averaging over one missing feature value per data point
- The submission dataset is missing nearly twice as much data, averaging nearly two missing feature values per data point
- There are no duplicate rows in the training dataset
- D_h and chf_exp seem to have some significant outliers on the upper end

### Renaming Featurse + Creating lists of columns by feature type

In [ ]:
# renaming columns to something more succinct and readable
column_renaming_dict = {'pressure [MPa]': 'pressure',
                        'mass_flux [kg/m2-s]': 'mass_flux',
                        'x_e_out [-]': 'x_e_out',
                        'D_e [mm]': 'D_e',
                        'D_h [mm]': 'D_h',
                        'length [mm]': 'length',
                        'chf_exp [MW/m2]': 'chf_exp'}

train = train.rename(columns=column_renaming_dict)
submission = submission.rename(columns=column_renaming_dict)
display(train.head())
                        
# creating groups by feature type
features = {'continuous': ['pressure', 'mass_flux', 'D_e', 'D_h', 'length', 'chf_exp'],
            'categorical': ['author', 'geometry']}

### Checking target distribution

In [ ]:
fix, ax = plt.subplots(figsize=(6, 6))
sns.kdeplot(data=train, x='x_e_out', fill=True, ax=ax).set_title('Target Distribution on Train Set');
ax = np.ravel(ax)
ax[0].grid(visible=True)

In [ ]:
# creating a log transformation of the target
transformed_target = np.power(10, train[['x_e_out']]) - 1

# plotting distribution
fix, ax = plt.subplots(figsize=(6, 6))
sns.kdeplot(data=transformed_target, x='x_e_out', fill=True, ax=ax).set_title('Transformed Target Distribution on Train Set [10^x - 1]');
ax = np.ravel(ax)
ax[0].grid(visible=True)

# adding this to the dataframe
train['log_x_e_out'] = transformed_target

#### 💡Insights: Target Distribution
- The target distribution on the training data has some left skewness, but it does not seem too severe
- Transforming the target with 10^x seems to make the distribution much more Gaussian. It may be worth looking into this to see if it helps the predictions

### Checking Feature Distribution

In [ ]:
# plotting distribution of each continuous feature in train and test datasets
fig, ax = plt.subplots(2, 3, figsize=(20, 10))
ax = np.ravel(ax)
palette = sns.color_palette('coolwarm', 2)

for i, col in enumerate(features['continuous']):
    sns.kdeplot(data=train, x=train[col], ax=ax[i], label='Train', color=palette[0], fill=True)
    sns.kdeplot(data=submission, x=submission[col], ax=ax[i], label='Test', color=palette[1], fill=True)
    ax[i].set_title(f'{col}', fontsize=12)
    ax[i].legend(title='Dataset', loc='upper right', labels=['Train', 'Test'])
    
fig.suptitle('Continuous Feature Distributions (Train & Test)', fontsize=20);
fig.tight_layout(pad=3)

In [ ]:
# creating function to create a distribution histogram for each discrete value
def create_dist_barplot(train_df, test_df, feature_name, ax):
    train_value_counts = pd.DataFrame(train_df.value_counts(feature_name, normalize=True))
    train_value_counts['Distribution'] = ['Train'] * train_value_counts.shape[0]
    test_value_counts = pd.DataFrame(test_df.value_counts(feature_name, normalize=True))
    test_value_counts['Distribution'] = ['Test'] * test_value_counts.shape[0]
    barplot_df = pd.concat([train_value_counts, test_value_counts], axis=0)
    barplot_df = barplot_df.rename(columns={'proportion': 'Density'})
    barplot_df = barplot_df.reset_index()
    sns.barplot(data=barplot_df, x=feature_name, y='Density', hue='Distribution', ax=ax, palette='coolwarm')

# plotting distribution of each integer feature in train and test datasets
fig, ax = plt.subplots(1, 2, figsize=(20, 8))
ax = np.ravel(ax)
palette = sns.color_palette('coolwarm', 2)

for i, col in enumerate(features['categorical']):
    create_dist_barplot(train, submission, col, ax[i])
    ax[i].set_title(f'{col}', fontsize=14)
    
fig.suptitle('Categorical Feature Distributions (Train & Test)', fontsize=20);
fig.tight_layout(pad=1)

#### 💡Insights: Feature Distributions:
- The feature distributions between the train and test datasets seem to be extremely similar for both categorical and numerical features

In [ ]:
# adjusting distributions of skewed features
train['D_e'] = np.log1p(train['D_e'])
train['D_h'] = np.log1p(train['D_h'])
train['length'] = np.log1p(train['length'])
train['chf_exp'] = np.log1p(train['chf_exp'])

submission['D_e'] = np.log1p(submission['D_e'])
submission['D_h'] = np.log1p(submission['D_h'])
submission['length'] = np.log1p(submission['length'])
submission['chf_exp'] = np.log1p(submission['chf_exp'])

# plotting distribution of each continuous feature in train and test datasets
fig, ax = plt.subplots(2, 3, figsize=(20, 10))
ax = np.ravel(ax)
palette = sns.color_palette('coolwarm', 2)

for i, col in enumerate(features['continuous']):
    sns.kdeplot(data=train, x=train[col], ax=ax[i], label='Train', color=palette[0], fill=True)
    sns.kdeplot(data=submission, x=submission[col], ax=ax[i], label='Test', color=palette[1], fill=True)
    ax[i].set_title(f'{col}', fontsize=12)
    ax[i].legend(title='Dataset', loc='upper right', labels=['Train', 'Test'])
    
fig.suptitle('Continuous Feature Distributions (Train & Test)', fontsize=20);
fig.tight_layout(pad=3)

### Examining Feature Correlation

In [ ]:
# calculating the raw correlation matrix
raw_correlation = train[features['continuous'] + ['x_e_out']].corr()

# only keeping the lower diagonal
correlation = raw_correlation.copy()
mask = np.zeros_like(correlation, dtype=bool)
mask[np.triu_indices_from(mask)] = True
correlation[mask] = np.nan

# plotting
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(correlation, annot=True, cmap='coolwarm', xticklabels=True, yticklabels=True, ax=ax, vmin=-1, vmax=1).set_title('Correlation Matrix', fontsize=20);

In [ ]:
# showing pairplot for continuous features
pairplot = sns.pairplot(data=train, vars=features['continuous'], diag_kind='kde');
pairplot.fig.suptitle('Pairplot for Continuous Features on Train Data', y=1.03, fontsize=20);

#### 💡Insights: Feature Correlation
- After transforming some of the features, *D_e* and *D_h* are highly correlated. Before transformation, there weren't any highly correlated features
- *D_h* is somewhat negatively correlated to pressure and positively correlated to *D_e*
- *D_e* is also somewhat negatively correlated to pressure, which makes sense considering it is positively correlated with *D_h*
- Looking at the pairplots, there is a strange correlation between *D_e* and *D_h*. They appear to have a perfect linear correlation with some random noise sprinkled in...

# 📏 Feature Engineering 📐

### Imputing missing categorical feature values and testing encoding technique

In [ ]:
# add missing labels using the mode for each the categories
train_imputed = train.copy(deep=True)
train_imputed['author'] = train_imputed['author'].replace(np.nan, 'Thompson')
train_imputed['geometry'] = train_imputed['geometry'].replace(np.nan, 'tube')

submission_imputed = submission.copy(deep=True)
submission_imputed['author'] = submission_imputed['author'].replace(np.nan, 'Thompson')
submission_imputed['geometry'] = submission_imputed['geometry'].replace(np.nan, 'tube')

# show the difference
display(train.head())
display(train_imputed.head())

# create a categorical one-hot encoder
categorical_transformer = ColumnTransformer(transformers=[('onehot1', OneHotEncoder(sparse_output=False), ['author']),
                                                          ('onehot2', OneHotEncoder(sparse_output=False), ['geometry']),
                                                          ('passthrough', 'passthrough', features['continuous'])],
                                            verbose_feature_names_out=False)
categorical_transformer.set_output(transform='pandas')

# pass the data through the encoder
train_cat_onehot_test = categorical_transformer.fit_transform(train_imputed)
display(train_cat_onehot_test.head())

### Creating Linear Regression based Imputer for D_e and D_h since they are highly correlated

In [ ]:
# creating a custom transformer
class D_transformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.D_e_regressor = LinearRegression()
        self.D_h_regressor = LinearRegression()
        
    def _impute_D_e(self, row):
        D_e = row['D_e']
        D_h = row['D_h']
        if np.isnan(D_e) and np.isnan(D_h)==False:
            D_e = self.D_e_regressor.predict(np.reshape(np.array(D_h), (-1, 1)))
        return float(D_e)

    def _impute_D_h(self, row):
        D_e = row['D_e']
        D_h = row['D_h']
        if np.isnan(D_h) and np.isnan(D_e)==False:
            D_h = self.D_h_regressor.predict(np.reshape(np.array(D_e), (-1, 1)))
        return float(D_h)
    
    def fit(self, X, y=None):
        # gathering D_e and D_h data where both are not NaN
        complete_D_data = X[['D_h', 'D_e']]
        filtered_D_data = complete_D_data[complete_D_data.isna().T.any() == False]

        D_e_array = np.reshape(np.array(filtered_D_data['D_e']), (-1, 1))
        D_h_array = np.reshape(np.array(filtered_D_data['D_h']), (-1, 1))

        # fitting regressors for each based on complete data
        self.D_e_regressor.fit(D_h_array, D_e_array)
        self.D_h_regressor.fit(D_e_array, D_h_array)
        
        return self
        
    def transform(self, X, y=None):
        X['D_e'] = X.apply(lambda row: self._impute_D_e(row), axis=1)
        X['D_h'] = X.apply(lambda row: self._impute_D_h(row), axis=1)
        
        return X

### Splitting "train" data into Train and Test Sets

In [ ]:
target_name = 'x_e_out'
features_to_include = features['continuous'] + features['categorical']

# splitting training data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(train_imputed[features_to_include],
                                                    train_imputed[target_name],
                                                    train_size=0.80,
                                                    shuffle=True,
                                                    random_state=1)

print(f'Size of X_train: {X_train.shape}\nSize of y_train: {y_train.shape}')
print(f'Size of X_test: {X_test.shape}\nSize of y_test: {y_test.shape}')

### Building a baseline model and assessing features

In [ ]:
# start with XGBoost
X_baseline = X_train
y_baseline = y_train

# create a categorical one-hot encoder
categorical_transformer = ColumnTransformer(transformers=[('onehot1', OneHotEncoder(sparse_output=False), ['author']),
                                                          ('onehot2', OneHotEncoder(sparse_output=False), ['geometry']),
                                                          ('passthrough', 'passthrough', features['continuous'])],
                                            verbose_feature_names_out=False)
categorical_transformer.set_output(transform='pandas')

# create a regression imputer for D_h and D_e
D_imputer = D_transformer()

# create an imputer
simple_imputer = SimpleImputer(strategy='median', copy=True)
simple_imputer.set_output(transform='pandas')

# create a baseline model to compare with
baseline_model = XGBRegressor(gamma=0.04, reg_lambda=0.04) # quickly add in some regularization by trial and error to prevent extreme overfitting

# create the pipeline
baseline_pipeline = Pipeline([('cat_transformer', categorical_transformer),
                              ('D_imputer', D_imputer),
                              ('imputer', simple_imputer),
                              ('regressor', baseline_model)])
cv_results = cross_validate(baseline_pipeline, X_baseline, y_baseline, cv=10, scoring='neg_root_mean_squared_error', return_train_score=True)
mean_test_score = np.mean(cv_results['test_score'])
train_test_score_rmse = np.std(cv_results['test_score']) # helpful to measure variance

# print scoring metrics
print('Test Scores on K-Folds: ' + str(np.round(cv_results['test_score'], decimals=3)))
print('Train Scores on K-Folds: ' + str(np.round(cv_results['train_score'], decimals=3)))
print(f'Mean Test Score: {np.round(mean_test_score, decimals=5)}')
print(f'Test Score K-Fold Std: {np.round(train_test_score_rmse, decimals=5)}')

In [ ]:
# add a feature of random noise to help judge feature importances
X_baseline['random_noise'] = np.random.normal(size=X_baseline.shape[0])

# create a categorical one-hot encoder that includes random noise
categorical_transformer_noise = ColumnTransformer(transformers=[('onehot1', OneHotEncoder(sparse_output=False), ['author']),
                                                                ('onehot2', OneHotEncoder(sparse_output=False), ['geometry']),
                                                                ('passthrough', 'passthrough', features['continuous'] + ['random_noise'])],
                                            verbose_feature_names_out=False)
categorical_transformer_noise.set_output(transform='pandas')

# create a baseline model to compare with
feature_importance_model = XGBRegressor(gamma=0.04, reg_lambda=0.04) # quickly add in some regularization by trial and error to prevent extreme overfitting

# create the pipeline
feature_importance_pipeline = Pipeline([('cat_transformer', categorical_transformer_noise),
                                        ('imputer', simple_imputer),
                                        ('regressor', feature_importance_model)])

# plotting feature importances
fig, ax = plt.subplots(figsize=(10, 7))
feature_importance_pipeline.fit(X_baseline, y_baseline)
plot_importance(feature_importance_model, ax=ax, importance_type='gain', show_values=False, xlabel='Gain', max_num_features=30);

In [ ]:
# create a partial dependence plot figure
n_cols = 3
n_rows = int(np.ceil(len(features['continuous']) / 3))
fig, ax = plt.subplots(n_rows, n_cols, figsize=(20, 5 * n_rows))
ax = np.ravel(ax)
for i in range(len(ax)): # hiding any unused axes
    if i >= len(features_to_include):
        ax[i].set_visible(False)
ax = ax[0:len(features_to_include)]

# transforming data
pdp_plot_data = categorical_transformer.fit_transform(X_train)
pdp_plot_data = simple_imputer.fit_transform(pdp_plot_data)

# adding title
fig.suptitle('Individual Conditional Expectation Plots for Features', fontsize=20);
fig.tight_layout(pad=3)

# plot PDP's and ICE's
baseline_pipeline.fit(X_baseline, y_baseline)
PartialDependenceDisplay.from_estimator(baseline_model, pdp_plot_data, features['continuous'], n_cols=n_cols, kind='both', subsample=100, ax=ax)

# adjusting y-axis values
for axis in ax:
    axis.set_ylim([-0.15, 0.15])

#### 💡Insights: Baseline Model
- Imputing missing categorical features with the mode seemed to work well
- Imputing missing continuous features with the median seemed to work well also
- The baseline XGBoost score seemed to perform fairly well and had much less variance than expected
- The most important feature by far seems to be *chf_exp*, which out of all features was highest correlated to the target. *pressure*, *D_e*, *D_h*, *length*, and some *author* one-hots follow.
- A random noise feature was added to use as a reference for how useful features are. All of the continuous features were above this threshold, but some *author* one-hots and all of the *geometry* one-hots were below the threshold.
- *geometry* may be a good candidate as a feature to drop
- The ICE plots show some odd behavior with the *D_e* and *D_h* features and their effect on the target.
- *chf_exp* and *pressure* seem to have a visible negative correlation with the target. *length* seems to have a less apparent positive correlation. This matches the Pearson coefficients calculated earlier

### Testing alternative imputation techniques

In [ ]:
# create a categorical one-hot encoder
categorical_transformer = ColumnTransformer(transformers=[('onehot1', OneHotEncoder(sparse_output=False), ['author']),
                                                          ('onehot2', OneHotEncoder(sparse_output=False), ['geometry']),
                                                          ('passthrough', 'passthrough', features['continuous'])],
                                            verbose_feature_names_out=False)
categorical_transformer.set_output(transform='pandas')

# create an imputer
knn_imputer = KNNImputer(n_neighbors=3, weights='uniform', copy=True)
knn_imputer.set_output(transform='pandas')
iterative_imputer = IterativeImputer(max_iter=10, imputation_order='descending')
iterative_imputer.set_output(transform='pandas')

# create a baseline model to compare with
impute_trial_model = XGBRegressor(gamma=0.04, reg_lambda=0.04) # quickly add in some regularization by trial and error to prevent extreme overfitting

# create the pipeline
impute_trial_pipeline = Pipeline([('cat_transformer', categorical_transformer),
                                  ('D_imputer', D_imputer),
                                  ('imputer', knn_imputer),
                                  ('regressor', impute_trial_model)])
cv_results = cross_validate(impute_trial_pipeline, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error', return_train_score=True)
mean_test_score = np.mean(cv_results['test_score'])
train_test_score_rmse = np.std(cv_results['test_score']) # helpful to measure variance

# print scoring metrics
print('Test Scores on K-Folds: ' + str(np.round(cv_results['test_score'], decimals=3)))
print('Train Scores on K-Folds: ' + str(np.round(cv_results['train_score'], decimals=3)))
print(f'Mean Test Score: {np.round(mean_test_score, decimals=5)}')
print(f'Test Score K-Fold Std: {np.round(train_test_score_rmse, decimals=5)}')

In [ ]:
# trying XGBoost native missing values functionality
impute_trial_pipeline = Pipeline([('cat_transformer', categorical_transformer),
                                  ('regressor', impute_trial_model)])
cv_results = cross_validate(impute_trial_pipeline, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error', return_train_score=True)
mean_test_score = np.mean(cv_results['test_score'])
train_test_score_rmse = np.std(cv_results['test_score']) # helpful to measure variance

# print scoring metrics
print('Test Scores on K-Folds: ' + str(np.round(cv_results['test_score'], decimals=3)))
print('Train Scores on K-Folds: ' + str(np.round(cv_results['train_score'], decimals=3)))
print(f'Mean Test Score: {np.round(mean_test_score, decimals=5)}')
print(f'Test Score K-Fold Std: {np.round(train_test_score_rmse, decimals=5)}')

#### 💡Insights: Alternative Imputation Techniques
- KNN Imputing seems to result in better performance for some values of K. The training takes significantly longer, though (on the order of 5-10 times as long). Running this through a hyperparameter optimizer may be problematic.
- Iterative Imputing does not improve results after some experimentation with hyperparameters
- XGBoost native functionality for missing values performed better than the baseline with statistical imputation techniques

### Exploring effect of dropping feature

In [ ]:
# dropping any features that are not needed
features_to_exclude = ['geometry']
all_features = features['continuous'] + features['categorical']
features_to_include = list(set(all_features) - set(features_to_exclude))

# splitting training data into train and test sets
X_train_drop, X_test_drop, y_train_drop, y_test_drop = train_test_split(train[features_to_include],
                                                                        train[target_name],
                                                                        train_size=0.80,
                                                                        shuffle=True,
                                                                        random_state=1)

print(f'Size of X_train: {X_train.shape}\nSize of y_train: {y_train.shape}')
print(f'Size of X_test: {X_test.shape}\nSize of y_test: {y_test.shape}')

In [ ]:
# create a categorical one-hot encoder
categorical_transformer_drop = ColumnTransformer(transformers=[('onehot1', OneHotEncoder(sparse_output=False), ['author']),
                                                               ('passthrough', 'passthrough', features['continuous'])],
                                                 verbose_feature_names_out=False)
categorical_transformer_drop.set_output(transform='pandas')

# create a baseline model to compare with
drop_model = XGBRegressor(gamma=0.04, reg_lambda=0.04) # quickly add in some regularization by trial and error to prevent extreme overfitting

# create the pipeline
drop_pipeline = Pipeline([('cat_transformer', categorical_transformer_drop),
                          ('D_imputer', D_imputer),
                          ('imputer', simple_imputer),
                          ('regressor', drop_model)])
cv_results = cross_validate(drop_pipeline, X_train_drop, y_train_drop, cv=10, scoring='neg_root_mean_squared_error', return_train_score=True)
mean_test_score = np.mean(cv_results['test_score'])
train_test_score_rmse = np.std(cv_results['test_score']) # helpful to measure variance

# print scoring metrics
print('Test Scores on K-Folds: ' + str(np.round(cv_results['test_score'], decimals=3)))
print('Train Scores on K-Folds: ' + str(np.round(cv_results['train_score'], decimals=3)))
print(f'Mean Test Score: {np.round(mean_test_score, decimals=5)}')
print(f'Test Score K-Fold Std: {np.round(train_test_score_rmse, decimals=5)}')

In [ ]:
# plotting feature importances
fig, ax = plt.subplots(figsize=(10, 7))
drop_pipeline.fit(X_train_drop, y_train_drop)
plot_importance(drop_model, ax=ax, importance_type='gain', show_values=False, xlabel='Gain', max_num_features=30);

#### 💡Insights: Feature Refinement
- Removing *geometry* feature slightly improved both score and variance. It is likely beneficial to completely remove this feature

### Trial to explore the results of using a transformed target

In [ ]:
# transforming the target
transformed_target = np.power(10, train['x_e_out']) - 1

# dropping any features that are not needed
features_to_include = features['continuous'] + features['categorical']

# splitting training data into train and test sets
X_train_trans, X_test_trans, y_train_trans, y_test_trans = train_test_split(train_imputed[features_to_include],
                                                                            transformed_target,
                                                                            train_size=0.80,
                                                                            shuffle=True,
                                                                            random_state=1)

print(f'Size of X_train: {X_train.shape}\nSize of y_train: {y_train.shape}')
print(f'Size of X_test: {X_test.shape}\nSize of y_test: {y_test.shape}')

In [ ]:
# creating a function for the scoring metric (not an sklearn out-of-the-box metric)
def transformed_root_mean_squared_error(y_true, y_pred):
    y_true_rev_trans = np.log10(y_true + 1)
    y_pred_rev_trans = np.log10(y_pred + 1)
    score = mean_squared_error(y_true_rev_trans, y_pred_rev_trans, squared=False)
    
    return score

# create a scorer object to use in sklearn functions
transformed_rmse = make_scorer(score_func=transformed_root_mean_squared_error, greater_is_better=False)

In [ ]:
# create a categorical one-hot encoder
categorical_transformer = ColumnTransformer(transformers=[('onehot1', OneHotEncoder(sparse_output=False), ['author']),
                                                          ('onehot2', OneHotEncoder(sparse_output=False), ['geometry']),
                                                          ('passthrough', 'passthrough', features['continuous'])],
                                            verbose_feature_names_out=False)
categorical_transformer.set_output(transform='pandas')

# create an imputer
imputer = SimpleImputer(strategy='median', copy=True)
imputer.set_output(transform='pandas')

# create a baseline model to compare with
trans_model = XGBRegressor(gamma=0.07, reg_lambda=0.07) # quickly add in some regularization by trial and error to prevent extreme overfitting

# create the pipeline
trans_pipeline = Pipeline([('cat_transformer', categorical_transformer),
                           ('D_imputer', D_imputer),
                           ('imputer', simple_imputer),
                           ('regressor', trans_model)])
cv_results = cross_validate(trans_pipeline, X_train_trans, y_train_trans, cv=10, scoring=transformed_rmse, return_train_score=True)
mean_test_score = np.mean(cv_results['test_score'])
train_test_score_rmse = np.std(cv_results['test_score']) # helpful to measure variance

# print scoring metrics
print('Test Scores on K-Folds: ' + str(np.round(cv_results['test_score'], decimals=3)))
print('Train Scores on K-Folds: ' + str(np.round(cv_results['train_score'], decimals=3)))
print(f'Mean Test Score: {np.round(mean_test_score, decimals=5)}')
print(f'Test Score K-Fold Std: {np.round(train_test_score_rmse, decimals=5)}')

#### 💡Insights: Transformed Target
- Transforming the target did seem to help the score slightly, although it was marginal and the regularization parameters had to be adjusted for the score to improve.
- Might try investigating later

### Training CatBoost, Random Forest, and LightGBM models with baseline imputer to see how the performance compares

In [ ]:
# resetting the categorical_transformer and imputer (getting rid of random noise feature and geometry)
categorical_transformer = ColumnTransformer(transformers=[('onehot1', OneHotEncoder(sparse_output=False), ['author']),
                                                          ('passthrough', 'passthrough', features['continuous'])],
                                            verbose_feature_names_out=False);
categorical_transformer.set_output(transform='pandas');

In [ ]:
# create a catboost model to compare with
cb_model = CatBoostRegressor(silent=True, allow_writing_files=False)

# create the pipeline
cb_pipeline = Pipeline([('cat_transformer', categorical_transformer),
                        ('D_imputer', D_imputer),
                        ('imputer', simple_imputer),
                        ('regressor', cb_model)])

cv_results = cross_validate(cb_pipeline, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error', return_train_score=True)
mean_test_score = np.mean(cv_results['test_score'])
train_test_score_rmse = np.std(cv_results['test_score']) # helpful to measure variance

# print scoring metrics
print('Test Scores on K-Folds: ' + str(np.round(cv_results['test_score'], decimals=3)))
print('Train Scores on K-Folds: ' + str(np.round(cv_results['train_score'], decimals=3)))
print(f'Mean Test Score: {np.round(mean_test_score, decimals=5)}')
print(f'Test Score K-Fold Std: {np.round(train_test_score_rmse, decimals=5)}')

In [ ]:
# create a random forest model to compare with
rf_model = RandomForestRegressor()

# create the pipeline
rf_pipeline = Pipeline([('cat_transformer', categorical_transformer),
                        ('D_imputer', D_imputer),
                        ('imputer', simple_imputer),
                        ('regressor', rf_model)])

cv_results = cross_validate(rf_pipeline, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error', return_train_score=True)
mean_test_score = np.mean(cv_results['test_score'])
train_test_score_rmse = np.std(cv_results['test_score']) # helpful to measure variance

# print scoring metrics
print('Test Scores on K-Folds: ' + str(np.round(cv_results['test_score'], decimals=3)))
print('Train Scores on K-Folds: ' + str(np.round(cv_results['train_score'], decimals=3)))
print(f'Mean Test Score: {np.round(mean_test_score, decimals=5)}')
print(f'Test Score K-Fold Std: {np.round(train_test_score_rmse, decimals=5)}')

In [ ]:
# create a LightGBM to compare with
lgbm_model = LGBMRegressor()

# create the pipeline
lgbm_pipeline = Pipeline([('cat_transformer', categorical_transformer),
                          ('D_imputer', D_imputer),
                          ('imputer', simple_imputer),
                          ('regressor', lgbm_model)])

cv_results = cross_validate(lgbm_pipeline, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error', return_train_score=True)
mean_test_score = np.mean(cv_results['test_score'])
train_test_score_rmse = np.std(cv_results['test_score']) # helpful to measure variance

# print scoring metrics
print('Test Scores on K-Folds: ' + str(np.round(cv_results['test_score'], decimals=3)))
print('Train Scores on K-Folds: ' + str(np.round(cv_results['train_score'], decimals=3)))
print(f'Mean Test Score: {np.round(mean_test_score, decimals=5)}')
print(f'Test Score K-Fold Std: {np.round(train_test_score_rmse, decimals=5)}')

#### 💡Insights: Alternative Models
- With default parameters, the CatBoost model had a better score and a comparable variance to the baseline model
- With default parameters, the Random Forest model had much worse score, but a lower variance than the baseline model
- With default parameters, the LightGBM model had an excellent score and a slightly worse variance than the baseline model
- All of these models are good contenders for the final model, and could be used for an ensemble. Random forest might not make the cut

### Training CatBoost and LightGBM models with native functionality for missing values to see how the performance compares

In [ ]:
# create the pipeline
cb_pipeline = Pipeline([('cat_transformer', categorical_transformer),
                        ('D_imputer', D_imputer),
                        ('regressor', cb_model)])

cv_results = cross_validate(cb_pipeline, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error', return_train_score=True)
mean_test_score = np.mean(cv_results['test_score'])
train_test_score_rmse = np.std(cv_results['test_score']) # helpful to measure variance

# print scoring metrics
print('Test Scores on K-Folds: ' + str(np.round(cv_results['test_score'], decimals=3)))
print('Train Scores on K-Folds: ' + str(np.round(cv_results['train_score'], decimals=3)))
print(f'Mean Test Score: {np.round(mean_test_score, decimals=5)}')
print(f'Test Score K-Fold Std: {np.round(train_test_score_rmse, decimals=5)}')

In [ ]:
# create the pipeline
lgbm_pipeline = Pipeline([('cat_transformer', categorical_transformer),
                          ('D_imputer', D_imputer),
                          ('regressor', lgbm_model)])

cv_results = cross_validate(lgbm_pipeline, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error', return_train_score=True)
mean_test_score = np.mean(cv_results['test_score'])
train_test_score_rmse = np.std(cv_results['test_score']) # helpful to measure variance

# print scoring metrics
print('Test Scores on K-Folds: ' + str(np.round(cv_results['test_score'], decimals=3)))
print('Train Scores on K-Folds: ' + str(np.round(cv_results['train_score'], decimals=3)))
print(f'Mean Test Score: {np.round(mean_test_score, decimals=5)}')
print(f'Test Score K-Fold Std: {np.round(train_test_score_rmse, decimals=5)}')

#### 💡Insights: Native Functionality for Missing Values
- The CatBoost model had a comparable score and a better variance than the simple imputation model
- The LightGBM model had a slightly better score and a better variance than the simple imputation model
- Both of these boosting models should use their native functionality as it improves their performance

# 🔧 Building and Optimizing an ML Model 🔨

### Optimizing Hyperparameters with Bayesian Optimization

In [ ]:
# optimize XGBoost hyperparameters with optuna
def objective(trial):
    # create the regressor object
    regressor = XGBRegressor(n_estimators=trial.suggest_int('n_estimators', 70, 300),
                             max_depth=trial.suggest_int('max_depth', 2, 8),
                             min_child_weight=trial.suggest_float('min_child_weight', 0, 6),
                             gamma=trial.suggest_float('gamma', 0.001, 6, log=True),
                             learning_rate=trial.suggest_float('learning_rate', 0.001, 0.3, log=True),
                             subsample=trial.suggest_float('subsample', 0.50, 1),
                             colsample_bytree=trial.suggest_float('colsample_bytree', 0.5, 1),
                             reg_lambda=trial.suggest_float('reg_lambda', 0.001, 5, log=True))

    # create lists to store scores
    train_scores = []
    test_scores = []
    
    # create the pipeline
    cv_pipeline = Pipeline([('cat_transformer', categorical_transformer),
                            ('D_imputer', D_imputer),
                            ('imputer', simple_imputer),
                            ('regressor', regressor)])
    
    # begin cross-validation
    cv_results = cross_validate(cv_pipeline, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error', return_train_score=True)
    mean_test_score = np.mean(cv_results['test_score'])
    train_test_score_rmse = np.std(cv_results['test_score']) # helpful to measure variance

    return mean_test_score

# begin optimization
optuna.logging.set_verbosity(optuna.logging.WARNING) # won't print progress for every single trial
cv_study = optuna.create_study(directions=['maximize'])
cv_study.optimize(objective, n_trials=20)
    
# get the n best trials
n = 10
study_results_zipped = [(t.values[0], t.params) for t in cv_study.get_trials()]
ordered_study_results = sorted(study_results_zipped, key=lambda x: x[0], reverse=True)
for i, t in enumerate(ordered_study_results):
    if i < n:
        print(f'Trial {i} Results:')
        print(f'Mean Test Score: {np.round(t[0], decimals=5)}')
        print(t[1])
        print('\n')

In [ ]:
# optimize CatBoost hyperparameters with optuna
def objective(trial):
    # create the regressor object
    regressor = CatBoostRegressor(iterations=trial.suggest_int('iterations', 100, 300),
                                  depth=trial.suggest_int('depth', 4, 10),
                                  l2_leaf_reg=trial.suggest_float('l2_leaf_reg', 0.001, 0.1, log=True),
                                  random_strength=trial.suggest_float('random_strength', 0.0001, 1, log=True),
                                  bagging_temperature=trial.suggest_float('bagging_temperature', 0, 1),
                                  min_data_in_leaf=trial.suggest_int('min_data_in_leaf', 1, 100),
                                  silent=True,
                                  allow_writing_files=False)

    # create lists to store scores
    train_scores = []
    test_scores = []
    
    # create imputer for catboost
    cb_imputer = ColumnTransformer(transformers=[('passthrough', 'passthrough', ['author']),
                                                 ('imputer', simple_imputer, features['continuous'])],
                                   verbose_feature_names_out=False);
    cb_imputer.set_output(transform='pandas');

    # create the pipeline
    cv_pipeline = Pipeline([('cat_transformer', categorical_transformer),
                            ('D_imputer', D_imputer),
                            ('imputer', simple_imputer),
                            ('regressor', regressor)])
    
    # begin cross-validation
    cv_results = cross_validate(cv_pipeline, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error', return_train_score=True)
    mean_test_score = np.mean(cv_results['test_score'])
    train_test_score_rmse = np.std(cv_results['test_score']) # helpful to measure variance

    return mean_test_score

# begin optimization
optuna.logging.set_verbosity(optuna.logging.WARNING) # won't print progress for every single trial
cv_study = optuna.create_study(directions=['maximize'])
cv_study.optimize(objective, n_trials=20)

# get the n best trials
n = 10
study_results_zipped = [(t.values[0], t.params) for t in cv_study.get_trials()]
ordered_study_results = sorted(study_results_zipped, key=lambda x: x[0], reverse=True)
for i, t in enumerate(ordered_study_results):
    if i < n:
        print(f'Trial {i} Results:')
        print(f'Mean Test Score: {np.round(t[0], decimals=5)}')
        print(t[1])
        print('\n')

In [ ]:
# optimize Random Forest hyperparameters with optuna
def objective(trial):
    # create the regressor object
    regressor = RandomForestRegressor(n_estimators=trial.suggest_int('n_estimators', 60, 250),
                                      max_depth=trial.suggest_categorical('max_depth', [10, 12, 15, 18, 20, 30, 40, 50, 70, 100, None]),
                                      min_samples_leaf=trial.suggest_int('min_samples_leaf', 1, 30),
                                      min_samples_split=trial.suggest_int('min_samples_split', 2, 20),
                                      n_jobs=-1)

    # create lists to store scores
    train_scores = []
    test_scores = []
    
    # create the pipeline
    cv_pipeline = Pipeline([('cat_transformer', categorical_transformer),
                            ('D_imputer', D_imputer),
                            ('imputer', simple_imputer),
                            ('regressor', regressor)])
    
    # begin cross-validation
    cv_results = cross_validate(cv_pipeline, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error', return_train_score=True)
    mean_test_score = np.mean(cv_results['test_score'])
    train_test_score_rmse = np.std(cv_results['test_score']) # helpful to measure variance

    return mean_test_score

# begin optimization
optuna.logging.set_verbosity(optuna.logging.WARNING) # won't print progress for every single trial
cv_study = optuna.create_study(directions=['maximize'])
cv_study.optimize(objective, n_trials=20)

# get the n best trials
n = 10
study_results_zipped = [(t.values[0], t.params) for t in cv_study.get_trials()]
ordered_study_results = sorted(study_results_zipped, key=lambda x: x[0], reverse=True)
for i, t in enumerate(ordered_study_results):
    if i < n:
        print(f'Trial {i} Results:')
        print(f'Mean Test Score: {np.round(t[0], decimals=5)}')
        print(t[1])
        print('\n')

In [ ]:
# optimize LightGBM hyperparameters with optuna
def objective(trial):
    # create the regressor object
    regressor = LGBMRegressor(n_estimators=trial.suggest_int('n_estimators', 70, 300),
                              learning_rate=trial.suggest_float('learning_rate', 0.001, 0.3, log=True),
                              num_leaves=trial.suggest_int('num_leaves', 20, 3000),
                              max_depth=trial.suggest_int('max_depth', 3, 12),
                              min_child_weight=trial.suggest_float('min_child_weight', 0.0005, 0.1, log=True),
                              min_child_samples=trial.suggest_int('min_child_samples', 5, 50),
                              subsample=trial.suggest_float('subsample', 0.5, 1),
                              colsample_bytree=trial.suggest_float('colsample_bytree', 0.5, 1),
                              reg_lambda=trial.suggest_float('reg_lambda', 0.001, 0.1, log=True))

    # create lists to store scores
    train_scores = []
    test_scores = []
    
    # create the pipeline
    cv_pipeline = Pipeline([('cat_transformer', categorical_transformer),
                            ('D_imputer', D_imputer),
                            ('imputer', simple_imputer),
                            ('regressor', regressor)])
    
    # begin cross-validation
    cv_results = cross_validate(cv_pipeline, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error', return_train_score=True)
    mean_test_score = np.mean(cv_results['test_score'])
    train_test_score_rmse = np.std(cv_results['test_score']) # helpful to measure variance

    return mean_test_score

# begin optimization
optuna.logging.set_verbosity(optuna.logging.WARNING) # won't print progress for every single trial
cv_study = optuna.create_study(directions=['maximize'])
cv_study.optimize(objective, n_trials=20)

# get the n best trials
n = 10
study_results_zipped = [(t.values[0], t.params) for t in cv_study.get_trials()]
ordered_study_results = sorted(study_results_zipped, key=lambda x: x[0], reverse=True)
for i, t in enumerate(ordered_study_results):
    if i < n:
        print(f'Trial {i} Results:')
        print(f'Mean Test Score: {np.round(t[0], decimals=5)}')
        print(t[1])
        print('\n')

### Creating finalized models with optimized hyperparameters and checking CV scores

In [ ]:
# create the XGBoost object
xgb_final_1 = XGBRegressor(n_estimators=267,
                           max_depth=7,
                           min_child_weight=5.93313,
                           gamma=0.002317,
                           learning_rate=0.034267,
                           subsample=0.60232,
                           colsample_bytree=0.62122,
                           reg_lambda=1.39154)

# create the pipeline
xgb_pipeline_1 = Pipeline([('cat_transformer', categorical_transformer),
                           ('D_imputer', D_imputer),
                           ('imputer', imputer),
                           ('regressor', xgb_final_1)])
cv_results = cross_validate(xgb_pipeline_1, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error', return_train_score=True)
mean_test_score = np.mean(cv_results['test_score'])
train_test_score_rmse = np.std(cv_results['test_score']) # helpful to measure variance

# print scoring metrics
print('Test Scores on K-Folds: ' + str(np.round(cv_results['test_score'], decimals=3)))
print('Train Scores on K-Folds: ' + str(np.round(cv_results['train_score'], decimals=3)))
print(f'Mean Test Score: {np.round(mean_test_score, decimals=5)}')
print(f'Test Score K-Fold Std: {np.round(train_test_score_rmse, decimals=5)}')

In [ ]:
# create the XGBoost object
xgb_final_2 = XGBRegressor(n_estimators=245,
                           max_depth=7,
                           min_child_weight=5.88154,
                           gamma=0.0024124,
                           learning_rate=0.035098,
                           subsample=0.62636,
                           colsample_bytree=0.61926,
                           reg_lambda=1.30091)

# create the pipeline
xgb_pipeline_2 = Pipeline([('cat_transformer', categorical_transformer),
                           ('D_imputer', D_imputer),
                           ('imputer', imputer),
                           ('regressor', xgb_final_2)])
cv_results = cross_validate(xgb_pipeline_2, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error', return_train_score=True)
mean_test_score = np.mean(cv_results['test_score'])
train_test_score_rmse = np.std(cv_results['test_score']) # helpful to measure variance

# print scoring metrics
print('Test Scores on K-Folds: ' + str(np.round(cv_results['test_score'], decimals=3)))
print('Train Scores on K-Folds: ' + str(np.round(cv_results['train_score'], decimals=3)))
print(f'Mean Test Score: {np.round(mean_test_score, decimals=5)}')
print(f'Test Score K-Fold Std: {np.round(train_test_score_rmse, decimals=5)}')

In [ ]:
# create a CatBoost object
cb_final = CatBoostRegressor(iterations=500,
                             depth=10,
                             l2_leaf_reg=0.0718249,
                             silent=True,
                             allow_writing_files=False)

# cb_final = CatBoostRegressor(silent=True,
#                              allow_writing_files=False)

# create the pipeline
cb_pipeline = Pipeline([('cat_transformer', categorical_transformer),
                        ('D_imputer', D_imputer),
                        ('imputer', imputer),
                        ('regressor', cb_final)])
cv_results = cross_validate(cb_pipeline, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error', return_train_score=True)
mean_test_score = np.mean(cv_results['test_score'])
train_test_score_rmse = np.std(cv_results['test_score']) # helpful to measure variance

# print scoring metrics
print('Test Scores on K-Folds: ' + str(np.round(cv_results['test_score'], decimals=3)))
print('Train Scores on K-Folds: ' + str(np.round(cv_results['train_score'], decimals=3)))
print(f'Mean Test Score: {np.round(mean_test_score, decimals=5)}')
print(f'Test Score K-Fold Std: {np.round(train_test_score_rmse, decimals=5)}')

In [ ]:
# create a random forest object
rf_final = RandomForestRegressor(n_estimators=141, 
                                 max_depth=15,
                                 min_samples_leaf=12,
                                 min_samples_split=5,
                                 n_jobs=-1)

# create the pipeline
rf_pipeline = Pipeline([('cat_transformer', categorical_transformer),
                        ('D_imputer', D_imputer),
                        ('imputer', imputer),
                        ('regressor', rf_final)])
cv_results = cross_validate(rf_pipeline, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error', return_train_score=True)
mean_test_score = np.mean(cv_results['test_score'])
train_test_score_rmse = np.std(cv_results['test_score']) # helpful to measure variance

# print scoring metrics
print('Test Scores on K-Folds: ' + str(np.round(cv_results['test_score'], decimals=3)))
print('Train Scores on K-Folds: ' + str(np.round(cv_results['train_score'], decimals=3)))
print(f'Mean Test Score: {np.round(mean_test_score, decimals=5)}')
print(f'Test Score K-Fold Std: {np.round(train_test_score_rmse, decimals=5)}')

In [ ]:
# create the LightGBM object
lgbm_final_1 = LGBMRegressor(n_estimators=176,
                             learning_rate=0.031217,
                             num_leaves=2681,
                             max_depth=11,
                             min_child_weight=0.03876,
                             min_child_samples=47,
                             subsample=0.61635,
                             colsample_bytree=0.510339,
                             reg_lambda=0.0082346)

# create the pipeline
lgbm_pipeline_1 = Pipeline([('cat_transformer', categorical_transformer),
                            ('D_imputer', D_imputer),
                            ('imputer', imputer),
                            ('regressor', lgbm_final_1)])
cv_results = cross_validate(lgbm_pipeline_1, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error', return_train_score=True)
mean_test_score = np.mean(cv_results['test_score'])
train_test_score_rmse = np.std(cv_results['test_score']) # helpful to measure variance

# print scoring metrics
print('Test Scores on K-Folds: ' + str(np.round(cv_results['test_score'], decimals=3)))
print('Train Scores on K-Folds: ' + str(np.round(cv_results['train_score'], decimals=3)))
print(f'Mean Test Score: {np.round(mean_test_score, decimals=5)}')
print(f'Test Score K-Fold Std: {np.round(train_test_score_rmse, decimals=5)}')

In [ ]:
# create the LightGBM object
lgbm_final_2 = LGBMRegressor(n_estimators=223,
                             learning_rate=0.029477,
                             num_leaves=2618,
                             max_depth=10,
                             min_child_weight=0.028960,
                             min_child_samples=49,
                             subsample=0.63466,
                             colsample_bytree=0.52098,
                             reg_lambda=0.007196)

# create the pipeline
lgbm_pipeline_2 = Pipeline([('cat_transformer', categorical_transformer),
                            ('D_imputer', D_imputer),
                            ('imputer', imputer),
                            ('regressor', lgbm_final_2)])
cv_results = cross_validate(lgbm_pipeline_2, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error', return_train_score=True)
mean_test_score = np.mean(cv_results['test_score'])
train_test_score_rmse = np.std(cv_results['test_score']) # helpful to measure variance

# print scoring metrics
print('Test Scores on K-Folds: ' + str(np.round(cv_results['test_score'], decimals=3)))
print('Train Scores on K-Folds: ' + str(np.round(cv_results['train_score'], decimals=3)))
print(f'Mean Test Score: {np.round(mean_test_score, decimals=5)}')
print(f'Test Score K-Fold Std: {np.round(train_test_score_rmse, decimals=5)}')

### Creating an ensemble model

In [ ]:
# creating a voting ensemble from the models
voting_model = VotingRegressor(estimators=[('xgb_1', xgb_final_1), ('xgb_2', xgb_final_2), ('lgbm_1', lgbm_final_1), ('lgbm_2', lgbm_final_2)])
voting_pipeline = Pipeline([('cat_transformer', categorical_transformer),
                            ('D_imputer', D_imputer),
                            ('imputer', imputer),
                            ('regressor', voting_model)])

# begin cross-validation
cv_results = cross_validate(voting_pipeline, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error', return_train_score=True)
mean_test_score = np.mean(cv_results['test_score'])
train_test_score_rmse = np.std(cv_results['test_score']) # helpful to measure variance

# print scoring metrics
print('Test Scores on K-Folds: ' + str(np.round(cv_results['test_score'], decimals=3)))
print('Train Scores on K-Folds: ' + str(np.round(cv_results['train_score'], decimals=3)))
print(f'Mean Test Score: {np.round(mean_test_score, decimals=5)}')
print(f'Test Score K-Fold Std: {np.round(train_test_score_rmse, decimals=5)}')

# fit the pipeline to the training data and report score on test data
voting_pipeline.fit(X_train, y_train)
y_test_pred = voting_pipeline.predict(X_test)
model_score = mean_squared_error(y_test, y_test_pred, squared=False)
print(f'Test Data Score: {np.round(model_score, decimals=5)}')

In [ ]:
# creating a stacked ensemble from the models
stacked_model = StackingRegressor(estimators=[('xgb_1', xgb_final_1), ('xgb_2', xgb_final_2), ('lgbm_1', lgbm_final_1), ('lgbm_2', lgbm_final_2)], final_estimator=BayesianRidge())
stacked_pipeline = Pipeline([('cat_transformer', categorical_transformer),
                             ('D_imputer', D_imputer),
                             ('imputer', imputer),
                             ('regressor', stacked_model)])

# begin cross-validation
cv_results = cross_validate(stacked_pipeline, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error', return_train_score=True)
mean_test_score = np.mean(cv_results['test_score'])
train_test_score_rmse = np.std(cv_results['test_score']) # helpful to measure variance

# print scoring metrics
print('Test Scores on K-Folds: ' + str(np.round(cv_results['test_score'], decimals=3)))
print('Train Scores on K-Folds: ' + str(np.round(cv_results['train_score'], decimals=3)))
print(f'Mean Test Score: {np.round(mean_test_score, decimals=5)}')
print(f'Test Score K-Fold Std: {np.round(train_test_score_rmse, decimals=5)}')

# fit the pipeline to the training data and report score on test data
stacked_pipeline.fit(X_train, y_train)
y_test_pred = stacked_pipeline.predict(X_test)
model_score = mean_squared_error(y_test, y_test_pred, squared=False)
print(f'Test Data Score: {np.round(model_score, decimals=5)}')

# 📦 Submission 📦

### Make Predictions

In [ ]:
# make predictions
X_submission = submission_imputed[features['continuous'] + features['categorical']]
y_submission_pred = voting_pipeline.predict(X_submission)

# formatting predictions for submission file output
submission_df = pd.DataFrame({'id': submission_imputed['id'], 'x_e_out [-]': y_submission_pred})
display(submission_df.head())

# saving predictions to .csv file for submission
submission_df.to_csv('submission.csv', header=True, index=False)